# Economic Data - Time series analysis for businesses

- notebook under production
- nothing is really working at this time
- review when addressing specific business and economic analysis

In [50]:
#standard packages
import pandas as pd
import requests
import numpy as np
import math
from math import pi, sqrt
import matplotlib.pyplot as plt
import geopandas as gpd

from typing import Optional

# import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import datetime
import time
# from tqdm import tqdm, trange

#import functions
import os
import sys

sys.path.append(
    os.path.join(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
)

#censusdis
from collections import OrderedDict
import censusdis.data as ced
import censusdis.maps as cem
import censusdis.values as cev
import censusdis.geography as cgeo
from censusdis.states import STATE_MA
from censusdis.maps import ShapeReader, plot_us_boundary
import censusdis.maps as cmap
from censusdis.values import ALL_SPECIAL_VALUES # Make sure it is there.

#gis packages
import osmnx as ox
import logging as lg
# ox.settings.log_console=True #use cache to avoid overloading the server
# ox.settings.memory_cache=True #use cache to avoid overloading the server
# ox.settings.memory = 4294967296 #set memory cache to 4GB
from shapely.geometry import Point
import folium
import networkx as nx

## future libaries
# import contextily as cx
# import fiona
# from pandana.loaders import osm
# import momepy
# import missingno as msno
# from us import states
# import imageio

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
ox.__version__

pd.options.display.max_columns = None # display all columns
pd.options.display.max_rows = None # display all rows

pd.set_option('display.max_colwidth', None)

# Boston Metro Area 

## Gather Census Geometry and Data

In [8]:
#load processed excel file from folder
df = pd.read_excel('../data/census/econ/combine21_xlsx/combine21_processed.xlsx')
df.head()

,State FIPS,State,County FIPS,County,2017 NAICS Code,2017 NAICS Description,Total CBP and NES Establishments,CBP Establishments Percentage,NES Establishments Percentage,CBP Establishments,"CBP Employment, including March 12th",CBP Employment Noise Flag,CBP First Quarter Payroll (in thousands),CBP First Quarter Payroll Noise Flag,CBP Annual Payroll (in thousands),CBP Annual Payroll Noise Flag,NES Establishments,NES Receipts (in thousands),NES Receipts Noise Flag
0,0,United States,-,-,0,Total,36626124,22.2,77.8,8148606,128346299,G,2014037035,G,8278573947,G,28477518,1501874326,G
1,0,United States,-,-,11,"Agriculture, Forestry, Fishing and Hunting",278987,8.4,91.6,23308,165688,G,1769574,G,8519646,G,255679,13327651,G
2,0,United States,-,-,113,Forestry and Logging,50488,16.3,83.7,8249,49632,G,604265,G,2641848,G,42239,2691287,G
3,0,United States,-,-,1131,Timber Tract Operations,N,N,N,482,3583,G,59934,G,237211,G,N,N,N
4,0,United States,-,-,1132,Forest Nurseries and Gathering of Forest Products,N,N,N,195,920,G,10583,G,48103,G,N,N,N


In [9]:
# df.loc[df['2017 NAICS Description'] == 'Total']
df

In [11]:
# American Community Survey 5-Year Data
# https://www.census.gov/data/developers/data-sets/acs-5year.html
DATASET = "acs/acs5"

# The year we want data for.
YEAR = 2022
TOTAL_POPULATION_VARIABLE = "B01003_001E"

# The variables we are going to query.
VARIABLES = ["NAME", TOTAL_POPULATION_VARIABLE]

df_states = ced.download(
    DATASET,
    YEAR,
    VARIABLES,
    state=STATE_MA,
    county="*",
)

df_states.head()

,STATE,COUNTY,NAME,B01003_001E
0,25,001,"Barnstable County, Massachusetts",229436
1,25,003,"Berkshire County, Massachusetts",128763
2,25,005,"Bristol County, Massachusetts",576699
3,25,007,"Dukes County, Massachusetts",20543
4,25,009,"Essex County, Massachusetts",806103


In [26]:
help(ced.variables)

Help on VariableCache in module censusdis.impl.varcache object:

class VariableCache(builtins.object)
 |  VariableCache(*, variable_source: Optional[censusdis.impl.varsource.base.VariableSource] = None)
 |
 |  A cache of variables and groups.
 |
 |  This looks a lot like a :py:class:`~VariableSource` but it
 |  implements a cache in front of a :py:class:`~VariableSource`.
 |
 |  Users will rarely if ever need to construct one of these
 |  themselves. In almost all cases they will use the singleton
 |  `censusdis.censusdata.variables`.
 |
 |  Methods defined here:
 |
 |  __contains__(self, item: Tuple[str, int, str]) -> bool
 |      Magic method behind the `in` operator.
 |
 |  __getitem__(self, item: Tuple[str, int, str])
 |      Magic method behind the `[]` operator.
 |
 |  __init__(self, *, variable_source: Optional[censusdis.impl.varsource.base.VariableSource] = None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __iter__(self) -> Iterable[Tuple[str, 

In [62]:
from censusdis.states import STATE_MA

DATASET = "timeseries/bds"
# DATASET = "cps/basic/nov"
# YEAR = 2020
ced.variables.all_groups(DATASET, None)
# ced.variables.all_variables("timeseries/bds", 2021, None)

,DATASET,YEAR,GROUP,DESCRIPTION
0,timeseries/bds,None,BDSEAGE,Business Dynamics Statistics: Establishment Age: 1978-2021
1,timeseries/bds,None,BDSEAGEESIZE,Business Dynamics Statistics: Establishment Age by Establishment Size: 1978-2021
2,timeseries/bds,None,BDSEAGEIESIZE,Business Dynamics Statistics: Establishment Age by Initial Establishment Size: 1978-2021
3,timeseries/bds,None,BDSESIZE,Business Dynamics Statistics: Establishment Size: 1978-2021
4,timeseries/bds,None,BDSFAGE,Business Dynamics Statistics: Firm Age: 1978-2021
5,timeseries/bds,None,BDSFAGEFSIZE,Business Dynamics Statistics: Firm Age by Firm Size: 1978-2021
6,timeseries/bds,None,BDSFAGEIFSIZE,Business Dynamics Statistics: Firm Age by Initial Firm Size: 1978-2021
7,timeseries/bds,None,BDSFSIZE,Business Dynamics Statistics: Firm Size: 1978-2021
8,timeseries/bds,None,BDSGEO,Business Dynamics Statistics: Geography: 1978-2021
9,timeseries/bds,None,BDSIESIZE,Business Dynamics Statistics: Initial Establishment Size: 1978-2021


In [53]:
# cgeo.geo_path_snake_specs(DATASET, None)

{'010': ['us'],
 '040': ['state'],
 '050': ['state', 'county'],
 '310': ['metropolitan_statistical_area_micropolitan_statistical_area']}

In [56]:
# help(ced.download)

Help on function download in module censusdis.data:

download(dataset: str, vintage: Union[int, Literal['timeseries']], download_variables: Union[str, Iterable[str], NoneType] = None, *, group: Union[str, Iterable[str], NoneType] = None, leaves_of_group: Union[str, Iterable[str], NoneType] = None, set_to_nan: Union[bool, Iterable[int]] = True, skip_annotations: bool = True, with_geometry: bool = False, remove_water: bool = False, download_contained_within: Optional[Dict[str, Union[str, Iterable[str]]]] = None, area_threshold: float = 0.8, api_key: Optional[str] = None, variable_cache: Optional[ForwardRef('VariableCache')] = None, row_keys: Union[str, Iterable[str], NoneType] = None, **kwargs: Union[str, Iterable[str]]) -> Union[pandas.core.frame.DataFrame, geopandas.geodataframe.GeoDataFrame]
    Download data from the US Census API.

    This is the main API for downloading US Census data with the
    `censusdis` package. There are many examples of how to use
    this in the demo note

In [95]:
from census import Census
# from us import states

c = Census("781655d3b92f252d48d8acae0f5669dddf3cb9a4")
# c.timeseries.state_county(('NAME', 'B25034_010E'), '*','*')
# help(Census)

In [96]:
import requests
import pandas as pd

variables = ['NAME','NAICS_LABEL','YEAR','FIRM','ESTAB','EMP']

api_key = "781655d3b92f252d48d8acae0f5669dddf3cb9a4"
url = f"https://api.census.gov/data/timeseries/bds?\
    get={variables}&\
    for=county:001&\
    in=state:01&\
    time=2021&\
    NAICS=*"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    print(df.head())
else:
    print("Failed to fetch data:", response.status_code, response.text)
b


Failed to fetch data: 400 error: the 'get' argument must be a comma separated list of variable names


NameError: name 'b' is not defined

In [164]:
df_list = pd.read_html("https://api.census.gov/data/timeseries/bds/variables.html")
# df_list[0].drop(['Concept' #df.drop(['B', 'C'], axis=1)
df = df_list[0][['Name']]

# Prepend a comma to each entry except the first
df['Name'] = df['Name'].apply(lambda x: ',' + x if x != df['Name'].iloc[0] else x)

# Perform cumulative sum to concatenate strings
df['Name_cumsum'] = df['Name'].cumsum()
df = df[:-1]

all_variables = df['Name_cumsum'].iloc[-1]

all_variables


# ruro.cumsum()

'CBSA,COUNTY,DENOM,EAGE,EMP,EMPSZES,EMPSZESI,EMPSZFI,EMPSZFII,ESTAB,ESTABS_ENTRY,ESTABS_ENTRY_RATE,ESTABS_EXIT,ESTABS_EXIT_RATE,FAGE,FIRM,FIRMDEATH_EMP,FIRMDEATH_ESTABS,FIRMDEATH_FIRMS,for,GEO_ID,GEOCOMP,in,INDGROUP,INDLEVEL,JOB_CREATION,JOB_CREATION_BIRTHS,JOB_CREATION_CONTINUERS,JOB_CREATION_RATE,JOB_CREATION_RATE_BIRTHS,JOB_DESTRUCTION,JOB_DESTRUCTION_CONTINUERS,JOB_DESTRUCTION_DEATHS,JOB_DESTRUCTION_RATE,JOB_DESTRUCTION_RATE_DEATHS,METRO,NAICS,NATION,NET_JOB_CREATION,NET_JOB_CREATION_RATE,REALLOCATION_RATE,SECTOR,STATE,SUBSECTOR,SUMLEVEL,time,ucgid,YEAR'

In [185]:
df_names_explained = df_list[0][['Name','Label']]
df_names_explained                          

,Name,Label
0,CBSA,Geography
1,COUNTY,Geography
2,DENOM,(DHS) denominator
3,EAGE,Establishment age code
4,EMP,Number of employees
5,EMPSZES,Employment size of establishments code
6,EMPSZESI,Initial employment size of establishments code
7,EMPSZFI,Employment size of firms code
8,EMPSZFII,Initial employment size of firms code
9,ESTAB,Number of establishments


In [187]:
import requests

key = "781655d3b92f252d48d8acae0f5669dddf3cb9a4"
# variables = 'NAME,NAICS_LABEL,YEAR,FIRM,ESTAB,EMP'
variables = 'NAME,CBSA,COUNTY,DENOM,EAGE,EMP,EMPSZES,EMPSZESI,EMPSZFI,EMPSZFII,ESTAB,ESTABS_ENTRY,ESTABS_ENTRY_RATE,ESTABS_EXIT,ESTABS_EXIT_RATE,FAGE,FIRM,FIRMDEATH_EMP,FIRMDEATH_ESTABS,FIRMDEATH_FIRMS,GEO_ID,GEOCOMP,INDGROUP,INDLEVEL,JOB_CREATION,JOB_CREATION_BIRTHS,JOB_CREATION_CONTINUERS,JOB_CREATION_RATE,JOB_CREATION_RATE_BIRTHS,JOB_DESTRUCTION,JOB_DESTRUCTION_CONTINUERS,JOB_DESTRUCTION_DEATHS,JOB_DESTRUCTION_RATE,JOB_DESTRUCTION_RATE_DEATHS,METRO,NAICS,NAICS_LABEL,NATION,NET_JOB_CREATION,NET_JOB_CREATION_RATE,REALLOCATION_RATE,SECTOR,STATE,SUBSECTOR,SUMLEVEL,YEAR'
county = '*'
state = '25'
year = '2021'
naics = '*'

url = f'https://api.census.gov/data/timeseries/bds?get={variables}&for=county:{county}&in=state:{state}&time={year}&NAICS={naics}&key={key}'
# url = f'https://api.census.gov/data/timeseries/bds?get=NAME,NAICS_LABEL,YEAR,FIRM,ESTAB,EMP&for=county:*&in=state:25&time=2020,2021&NAICS=*&key={key}'
response = requests.get(url)
data = response.json()  # Convert JSON response to a list of lists

# Create a DataFrame from the JSON data
# The first element in data is the header, followed by data rows
df = pd.DataFrame(data[1:], columns=data[0])

df


,NAME,CBSA,COUNTY,DENOM,EAGE,EMP,EMPSZES,EMPSZESI,EMPSZFI,EMPSZFII,ESTAB,ESTABS_ENTRY,ESTABS_ENTRY_RATE,ESTABS_EXIT,ESTABS_EXIT_RATE,FAGE,FIRM,FIRMDEATH_EMP,FIRMDEATH_ESTABS,FIRMDEATH_FIRMS,GEO_ID,GEOCOMP,INDGROUP,INDLEVEL,JOB_CREATION,JOB_CREATION_BIRTHS,JOB_CREATION_CONTINUERS,JOB_CREATION_RATE,JOB_CREATION_RATE_BIRTHS,JOB_DESTRUCTION,JOB_DESTRUCTION_CONTINUERS,JOB_DESTRUCTION_DEATHS,JOB_DESTRUCTION_RATE,JOB_DESTRUCTION_RATE_DEATHS,METRO,NAICS,NAICS_LABEL,NATION,NET_JOB_CREATION,NET_JOB_CREATION_RATE,REALLOCATION_RATE,SECTOR,STATE,SUBSECTOR,SUMLEVEL,YEAR,time,NAICS,state,county
0,"Suffolk County, Massachusetts",None,025,650397,001,622582,001,001,001,001,19947,1871,9.238,2483,12.260,001,15543,10506,1355,1311,0500000US25025,00,None,2,61395,17943,43452,9.440,2.759,117660,88700,28960,18.090,4.453,A,00,Total for all sectors,None,-56265,-8.651,18.879,00,25,None,050,2021,2021,00,25,025
1,"Suffolk County, Massachusetts",None,025,2466,001,4931,001,001,001,001,1040,1040,200.000,0,0.000,010,1038,0,0,0,0500000US25025,00,None,2,4931,4931,0,200.000,200.000,0,0,0,0.000,0.000,A,00,Total for all sectors,None,4931,200.000,0.000,00,25,None,050,2021,2021,00,25,025
2,"Suffolk County, Massachusetts",None,025,35267,001,32991,001,001,001,001,3661,164,4.164,719,18.256,065,3623,2492,545,545,0500000US25025,00,None,2,7435,1430,6005,21.082,4.055,11867,8327,3540,33.649,10.038,A,00,Total for all sectors,None,-4432,-12.567,42.164,00,25,None,050,2021,2021,00,25,025
3,"Suffolk County, Massachusetts",None,025,41691,001,37959,001,001,001,001,2349,91,3.695,318,12.914,070,2291,2187,229,228,0500000US25025,00,None,2,4885,774,4111,11.717,1.857,12578,9661,2917,30.170,6.997,A,00,Total for all sectors,None,-7693,-18.453,23.435,00,25,None,050,2021,2021,00,25,025
4,"Suffolk County, Massachusetts",None,025,311547,001,302603,001,001,001,001,8319,291,3.381,867,10.073,075,6714,4177,495,470,0500000US25025,00,None,2,23718,4315,19403,7.613,1.385,41815,33260,8555,13.422,2.746,A,00,Total for all sectors,None,-18097,-5.809,15.226,00,25,None,050,2021,2021,00,25,025
5,"Suffolk County, Massachusetts",None,025,259427,001,244098,001,001,001,001,4578,285,6.032,579,12.254,150,1877,1650,86,68,0500000US25025,00,None,2,20247,6493,13754,7.805,2.503,51129,37181,13948,19.708,5.376,A,00,Total for all sectors,None,-30882,-11.904,15.609,00,25,None,050,2021,2021,00,25,025
6,"Suffolk County, Massachusetts",None,025,54668,001,53444,001,001,001,635,11406,1318,11.430,1568,13.598,001,11352,3996,1195,1184,0500000US25025,00,None,2,10073,3425,6648,18.426,6.265,12461,7201,5260,22.794,9.622,A,00,Total for all sectors,None,-2388,-4.368,36.852,00,25,None,050,2021,2021,00,25,025
7,"Suffolk County, Massachusetts",None,025,152653,001,140261,001,001,001,650,3605,144,3.936,252,6.887,001,2951,5509,139,122,0500000US25025,00,None,2,13466,4207,9259,8.821,2.756,38202,30419,7783,25.025,5.099,A,00,Total for all sectors,None,-24736,-16.204,17.643,00,25,None,050,2021,2021,00,25,025
8,"Suffolk County, Massachusetts",None,025,443077,001,428877,001,001,001,657,4936,409,8.078,663,13.095,001,1252,1001,21,5,0500000US25025,00,None,2,37806,10311,27495,8.533,2.327,66852,50935,15917,15.088,3.592,A,00,Total for all sectors,None,-29046,-6.556,17.065,00,25,None,050,2021,2021,00,25,025
9,"Suffolk County, Massachusetts",None,025,58194,001,54388,001,001,635,001,11575,1351,11.516,1665,14.192,001,11521,5770,1269,1249,0500000US25025,00,None,2,9617,4245,5372,16.526,7.295,17233,9537,7696,29.613,13.225,A,00,Total for all sectors,None,-7616,-13.087,33.052,00,25,None,050,2021,2021,00,25,025


In [75]:
dataset = "timeseries/bds"

vintage = 2021
state = STATE_MA
variables = ['EMP']
group = 'BDSNAICS'

df_data_BDSNAICS = ced.download(
    dataset,
    vintage,
    download_variables=None,  # This should not be None or empty
    state=state,
    county='*',
    skip_annotations = True,
    with_geometry = True,
)
df_data_BDSNAICS.head()


CensusApiException: Census API request to https://api.census.gov/data/timeseries/bds?get=&for=county&in=state%3A25 failed with status 400. error: the 'get' argument must be a comma separated list of variable names

In [60]:
group = 'BDSNAICS'
state = State_MA

df_data_BDSNAICS = ced.download(DATASET, None, state=STATE_MA, county='*')

CensusApiException: Census API request to https://api.census.gov/data/timeseries/bds?get=&for=county&in=state%3A25 failed with status 400. error: the 'get' argument must be a comma separated list of variable names

In [20]:
# American Community Survey 5-Year Data
# https://www.census.gov/data/developers/data-sets/acs-5year.html
DATASET_bds = "timeseries/bds"

# The year we want data for.
# YEAR_bds = 2021
# TOTAL_POPULATION_VARIABLE = "B01003_001E"

# The variables we are going to query.
VARIABLES_bds = ["JOB_CREATION"]

df_states_bds = ced.download(
    DATASET_bds,
    YEAR_bds,
    VARIABLES_bds,
    state=STATE_MA,
    # county="*",
)

df_states_bds.head()

GeoException: Census API request to https://api.census.gov/data/2021/timeseries/bds/geography.json failed with status 404. <!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>